In [ ]:
## Use keras embedding to encode the dataset and see how it performs agains glove

In [1]:
import pandas as pd
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [4]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [5]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [3]:
from sklearn import preprocessing
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

In [4]:
from sklearn.model_selection import train_test_split
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [16]:
ytrain

array([2, 0, 2, ..., 0, 2, 2])

In [5]:
#binarize the labels for neural net
from keras.utils import np_utils
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [19]:
xtrain

array([ 'Her hair was the brightest living gold, and despite the poverty of her clothing, seemed to set a crown of distinction on her head.',
       '"No," he said, "oh, no a member of my family my niece, and a most accomplished woman."',
       'The magistrate appeared at first perfectly incredulous, but as I continued he became more attentive and interested; I saw him sometimes shudder with horror; at others a lively surprise, unmingled with disbelief, was painted on his countenance.',
       ...,
       'The medical testimony spoke confidently of the virtuous character of the deceased.',
       'When we arrived, after a little rest, he led me over the house and pointed out to me the rooms which my mother had inhabited.',
       'Some were destroyed; the major part escaped by quick and well ordered movements; and danger made them careful.'], dtype=object)

In [6]:
vocab_size = 10000
xtrain_enc = [one_hot(d, vocab_size) for d in xtrain]
xvalid_enc = [one_hot(d, vocab_size) for d in xvalid]
#print(encoded_docs_train)

In [7]:
max_length = 300
padded_docs_train = pad_sequences(xtrain_enc, maxlen=max_length, padding='post')
padded_docs_valid = pad_sequences(xvalid_enc, maxlen=max_length, padding='post')

In [38]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 300, 300)          3000000   
_________________________________________________________________
flatten_5 (Flatten)          (None, 90000)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 270003    
Total params: 3,270,003
Trainable params: 3,270,003
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=5,  verbose=1,
         validation_data=(padded_docs_valid,yvalid_enc))
#model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=5,  verbose=1)

Train on 17621 samples, validate on 1958 samples
Epoch 1/5
17621/17621 [==============================] - 12s 691us/step - loss: 0.0161 - acc: 0.9988 - val_loss: 0.6198 - val_acc: 0.7962
Epoch 2/5
17621/17621 [==============================] - 12s 689us/step - loss: 0.0096 - acc: 0.9997 - val_loss: 0.6523 - val_acc: 0.7967
Epoch 3/5
17621/17621 [==============================] - 12s 677us/step - loss: 0.0062 - acc: 0.9998 - val_loss: 0.6789 - val_acc: 0.7932
Epoch 4/5
17621/17621 [==============================] - 12s 689us/step - loss: 0.0043 - acc: 1.0000 - val_loss: 0.7027 - val_acc: 0.7952
Epoch 5/5
17621/17621 [==============================] - 12s 681us/step - loss: 0.0030 - acc: 1.0000 - val_loss: 0.7262 - val_acc: 0.7921


In [9]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import EarlyStopping
from keras import optimizers

In [49]:
# make another model for comparison
# create a simple 3 layer sequential neural net
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
model.add(Flatten())

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 300, 300)          3000000   
_________________________________________________________________
flatten_9 (Flatten)          (None, 90000)             0         
_________________________________________________________________
dense_14 (Dense)             (None, 300)               27000300  
_________________________________________________________________
dropout_6 (Dropout)          (None, 300)               0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 300)               1200      
_________________________________________________________________
dense_15 (Dense)             (None, 300)               90300     
_________________________________________________________________
dropout_7 (Dropout)          (None, 300)               0         
__________

In [50]:
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=5,  verbose=1,
         validation_data=(padded_docs_valid,yvalid_enc))

Train on 17621 samples, validate on 1958 samples
Epoch 1/5
17621/17621 [==============================] - 75s 4ms/step - loss: 1.2875 - val_loss: 1.1930
Epoch 2/5
17621/17621 [==============================] - 74s 4ms/step - loss: 0.7970 - val_loss: 0.9204
Epoch 3/5
17621/17621 [==============================] - 74s 4ms/step - loss: 0.3205 - val_loss: 0.5748
Epoch 4/5
17621/17621 [==============================] - 74s 4ms/step - loss: 0.0965 - val_loss: 0.8387
Epoch 5/5
17621/17621 [==============================] - 74s 4ms/step - loss: 0.0300 - val_loss: 0.8989


In [52]:
# trying lstm with early stopping to compare

from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.layers.recurrent import LSTM, GRU

model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 300, 300)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_17 (Dense)             (None, 1024)              103424    
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dropout_9 (Dropout)          (None, 1024)              0         
__________

In [54]:
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 159s 9ms/step - loss: 1.0898 - val_loss: 1.0877
Epoch 2/100
17621/17621 [==============================] - 172s 10ms/step - loss: 1.0896 - val_loss: 1.0891
Epoch 3/100
17621/17621 [==============================] - 142s 8ms/step - loss: 1.0888 - val_loss: 1.0876
Epoch 4/100
17621/17621 [==============================] - 121s 7ms/step - loss: 1.0897 - val_loss: 1.0881
Epoch 5/100
17621/17621 [==============================] - 121s 7ms/step - loss: 1.0888 - val_loss: 1.0876
Epoch 6/100
17621/17621 [==============================] - 122s 7ms/step - loss: 1.0894 - val_loss: 1.0877


In [55]:
# less dropout
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 300, 300)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_20 (Dense)             (None, 1024)              103424    
_________________________________________________________________
dropout_10 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dropout_11 (Dropout)         (None, 1024)              0         
__________

In [56]:
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 123s 7ms/step - loss: 1.0894 - val_loss: 1.0887
Epoch 2/100
17621/17621 [==============================] - 123s 7ms/step - loss: 1.0887 - val_loss: 1.0876
Epoch 3/100
17621/17621 [==============================] - 122s 7ms/step - loss: 1.0880 - val_loss: 1.0879
Epoch 4/100
17621/17621 [==============================] - 122s 7ms/step - loss: 1.0881 - val_loss: 1.0877
Epoch 5/100
17621/17621 [==============================] - 123s 7ms/step - loss: 1.0883 - val_loss: 1.0876


In [59]:
# simpler model
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.3))

# model.add(Dense(1024, activation='relu'))

# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.3))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_23 (Dense)             (None, 3)                 303       
_________________________________________________________________
activation_6 (Activation)    (None, 3)                 0         
Total params: 3,160,703
Trainable params: 3,160,703
Non-trainable params: 0
_________________________________________________________________
None


In [60]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 119s 7ms/step - loss: 1.0896 - val_loss: 1.0887
Epoch 2/100
17621/17621 [==============================] - 118s 7ms/step - loss: 1.0889 - val_loss: 1.0880
Epoch 3/100
17621/17621 [==============================] - 116s 7ms/step - loss: 1.0880 - val_loss: 1.0877
Epoch 4/100
17621/17621 [==============================] - 116s 7ms/step - loss: 1.0878 - val_loss: 1.0880
Epoch 5/100
17621/17621 [==============================] - 116s 7ms/step - loss: 1.0879 - val_loss: 1.0881
Epoch 6/100
17621/17621 [==============================] - 118s 7ms/step - loss: 1.0879 - val_loss: 1.0876
Epoch 7/100
17621/17621 [==============================] - 119s 7ms/step - loss: 1.0877 - val_loss: 1.0876
Epoch 8/100
17621/17621 [==============================] - 120s 7ms/step - loss: 1.0875 - val_loss: 1.0875
Epoch 9/100
17621/17621 [==============================] - 122s 7ms/step - loss: 1.0880 - val_l

In [61]:
# add one more layer of dense to teh simpler model

model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

#model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.3))
model.add(LSTM(100))
model.add(BatchNormalization())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.3))
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_24 (Dense)             (None, 1024)              103424    
_________________________________________________________________
dense_25 (Dense)             (None, 3)                 3075      
_________________________________________________________________
activation_7 (Activation)    (None, 3)                 0         
Total params: 3,266,899
Trainable params: 3,266,899
Non-trainable params: 0
_________________________________________________________________
None


In [62]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 124s 7ms/step - loss: 1.0890 - val_loss: 1.0905
Epoch 2/100
17621/17621 [==============================] - 125s 7ms/step - loss: 1.0881 - val_loss: 1.0876
Epoch 3/100
17621/17621 [==============================] - 121s 7ms/step - loss: 1.0881 - val_loss: 1.0876
Epoch 4/100
17621/17621 [==============================] - 121s 7ms/step - loss: 1.0878 - val_loss: 1.0876
Epoch 5/100
17621/17621 [==============================] - 125s 7ms/step - loss: 1.0876 - val_loss: 1.0879
Epoch 6/100
17621/17621 [==============================] - 121s 7ms/step - loss: 1.0875 - val_loss: 1.0882


In [63]:
##lstms don't work so well here, why??
## so back to dense layers - later try convolutions

from keras.layers.core import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
model.add(Flatten())

model.add(Dense(300, input_dim=300, activation='relu'))
#model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
#model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
flatten_12 (Flatten)         (None, 90000)             0         
_________________________________________________________________
dense_26 (Dense)             (None, 300)               27000300  
_________________________________________________________________
batch_normalization_7 (Batch (None, 300)               1200      
_________________________________________________________________
dense_27 (Dense)             (None, 300)               90300     
_________________________________________________________________
batch_normalization_8 (Batch (None, 300)               1200      
_________________________________________________________________
dense_28 (Dense)             (None, 3)                 903       
__________

In [64]:
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=5,  verbose=1,
         validation_data=(padded_docs_valid,yvalid_enc))

Train on 17621 samples, validate on 1958 samples
Epoch 1/5
17621/17621 [==============================] - 75s 4ms/step - loss: 0.8608 - val_loss: 0.8716
Epoch 2/5
17621/17621 [==============================] - 73s 4ms/step - loss: 0.2487 - val_loss: 0.8537
Epoch 3/5
17621/17621 [==============================] - 73s 4ms/step - loss: 0.0700 - val_loss: 1.4063
Epoch 4/5
17621/17621 [==============================] - 73s 4ms/step - loss: 0.0326 - val_loss: 1.2074
Epoch 5/5
17621/17621 [==============================] - 73s 4ms/step - loss: 0.0255 - val_loss: 2.0335


In [65]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
model.add(Flatten())

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
flatten_13 (Flatten)         (None, 90000)             0         
_________________________________________________________________
dense_29 (Dense)             (None, 300)               27000300  
_________________________________________________________________
dropout_12 (Dropout)         (None, 300)               0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 300)               1200      
_________________________________________________________________
dense_30 (Dense)             (None, 300)               90300     
_________________________________________________________________
dropout_13 (Dropout)         (None, 300)               0         
__________

In [72]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 74s 4ms/step - loss: 0.0132 - val_loss: 1.1766
Epoch 2/100
17621/17621 [==============================] - 73s 4ms/step - loss: 0.0042 - val_loss: 1.2773
Epoch 3/100
17621/17621 [==============================] - 73s 4ms/step - loss: 0.0023 - val_loss: 1.2371
Epoch 4/100
17621/17621 [==============================] - 73s 4ms/step - loss: 0.0040 - val_loss: 1.3673


In [67]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 19s 1ms/step - loss: 0.0764 - val_loss: 0.8002
Epoch 2/100
17621/17621 [==============================] - 18s 1ms/step - loss: 0.0410 - val_loss: 0.9359
Epoch 3/100
17621/17621 [==============================] - 18s 1ms/step - loss: 0.0253 - val_loss: 0.9545
Epoch 4/100
17621/17621 [==============================] - 18s 1ms/step - loss: 0.0163 - val_loss: 1.0312


In [68]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=128, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 42s 2ms/step - loss: 0.0160 - val_loss: 0.9846
Epoch 2/100
17621/17621 [==============================] - 42s 2ms/step - loss: 0.0112 - val_loss: 1.1052
Epoch 3/100
17621/17621 [==============================] - 41s 2ms/step - loss: 0.0058 - val_loss: 1.1030
Epoch 4/100
17621/17621 [==============================] - 41s 2ms/step - loss: 0.0046 - val_loss: 1.1845


In [69]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
model.add(Flatten())

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
flatten_14 (Flatten)         (None, 90000)             0         
_________________________________________________________________
dense_33 (Dense)             (None, 300)               27000300  
_________________________________________________________________
dropout_15 (Dropout)         (None, 300)               0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 300)               1200      
_________________________________________________________________
dense_34 (Dense)             (None, 300)               90300     
_________________________________________________________________
dropout_16 (Dropout)         (None, 300)               0         
__________

In [70]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 75s 4ms/step - loss: 1.3059 - val_loss: 1.0996
Epoch 2/100
17621/17621 [==============================] - 73s 4ms/step - loss: 1.0832 - val_loss: 0.9484
Epoch 3/100
17621/17621 [==============================] - 73s 4ms/step - loss: 0.5884 - val_loss: 0.6740
Epoch 4/100
17621/17621 [==============================] - 73s 4ms/step - loss: 0.2412 - val_loss: 0.5939
Epoch 5/100
17621/17621 [==============================] - 73s 4ms/step - loss: 0.0781 - val_loss: 0.7839
Epoch 6/100
17621/17621 [==============================] - 73s 4ms/step - loss: 0.0329 - val_loss: 0.8665
Epoch 7/100
17621/17621 [==============================] - 73s 4ms/step - loss: 0.0212 - val_loss: 0.9947


In [71]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=32, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 153s 9ms/step - loss: 0.1156 - val_loss: 1.0272
Epoch 2/100
17621/17621 [==============================] - 153s 9ms/step - loss: 0.1005 - val_loss: 1.0019
Epoch 3/100
17621/17621 [==============================] - 153s 9ms/step - loss: 0.0534 - val_loss: 0.8861
Epoch 4/100
17621/17621 [==============================] - 153s 9ms/step - loss: 0.0398 - val_loss: 1.0639
Epoch 5/100
17621/17621 [==============================] - 153s 9ms/step - loss: 0.0349 - val_loss: 1.2094
Epoch 6/100
17621/17621 [==============================] - 153s 9ms/step - loss: 0.0376 - val_loss: 1.1328


In [73]:
## trying out cnn

from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [76]:
# create the model
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(Conv1D(filters=300, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 300, 300)          270300    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 150, 300)          0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 45000)             0         
_________________________________________________________________
dense_36 (Dense)             (None, 250)               11250250  
_________________________________________________________________
dense_37 (Dense)             (None, 3)                 753       
Total params: 14,521,303
Trainable params: 14,521,303
Non-trainable params: 0
________________________________________________________________

In [77]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 104s 6ms/step - loss: 0.7075 - acc: 0.6817 - val_loss: 0.5048 - val_acc: 0.7932
Epoch 2/100
17621/17621 [==============================] - 98s 6ms/step - loss: 0.3175 - acc: 0.8831 - val_loss: 0.5114 - val_acc: 0.7896
Epoch 3/100
17621/17621 [==============================] - 97s 5ms/step - loss: 0.1525 - acc: 0.9438 - val_loss: 0.6457 - val_acc: 0.7901
Epoch 4/100
17621/17621 [==============================] - 98s 6ms/step - loss: 0.0613 - acc: 0.9810 - val_loss: 0.9341 - val_acc: 0.7855


In [78]:
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(Conv1D(filters=300, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Flatten())

model.add(Dense(250, activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 300, 300)          270300    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 150, 300)          0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 150, 300)          0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 150, 300)          1200      
_________________________________________________________________
flatten_17 (Flatten)         (None, 45000)             0         
_________________________________________________________________
dense_38 (Dense)             (None, 250)               11250250  
__________

In [79]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 109s 6ms/step - loss: 0.9404 - acc: 0.5711 - val_loss: 0.9445 - val_acc: 0.4637
Epoch 2/100
17621/17621 [==============================] - 107s 6ms/step - loss: 0.3981 - acc: 0.8442 - val_loss: 0.5371 - val_acc: 0.7906
Epoch 3/100
17621/17621 [==============================] - 107s 6ms/step - loss: 0.2022 - acc: 0.9267 - val_loss: 0.5801 - val_acc: 0.7737
Epoch 4/100
17621/17621 [==============================] - 107s 6ms/step - loss: 0.1021 - acc: 0.9630 - val_loss: 0.6444 - val_acc: 0.7901
Epoch 5/100
17621/17621 [==============================] - 107s 6ms/step - loss: 0.0623 - acc: 0.9784 - val_loss: 0.7826 - val_acc: 0.7778


In [82]:
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(Conv1D(filters=8, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Flatten())

model.add(Dense(250, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_25 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 300, 8)            7208      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 150, 8)            0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 150, 8)            0         
_________________________________________________________________
batch_normalization_22 (Batc (None, 150, 8)            32        
_________________________________________________________________
flatten_20 (Flatten)         (None, 1200)              0         
_________________________________________________________________
dense_46 (Dense)             (None, 250)               300250    
__________

In [83]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 21s 1ms/step - loss: 1.0777 - acc: 0.5215 - val_loss: 0.9330 - val_acc: 0.6129
Epoch 2/100
17621/17621 [==============================] - 19s 1ms/step - loss: 0.5483 - acc: 0.7762 - val_loss: 0.5948 - val_acc: 0.7661
Epoch 3/100
17621/17621 [==============================] - 20s 1ms/step - loss: 0.3854 - acc: 0.8511 - val_loss: 0.5341 - val_acc: 0.7773
Epoch 4/100
17621/17621 [==============================] - 20s 1ms/step - loss: 0.3051 - acc: 0.8839 - val_loss: 0.5730 - val_acc: 0.7799
Epoch 5/100
17621/17621 [==============================] - 20s 1ms/step - loss: 0.2449 - acc: 0.9073 - val_loss: 0.6373 - val_acc: 0.7783
Epoch 6/100
17621/17621 [==============================] - 20s 1ms/step - loss: 0.2169 - acc: 0.9168 - val_loss: 0.6588 - val_acc: 0.7814


In [84]:
from keras import optimizers

model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(Conv1D(filters=8, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Flatten())

model.add(Dense(250, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(3, activation='softmax'))

nadam = optimizers.Nadam (lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 300, 8)            7208      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 150, 8)            0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 150, 8)            0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 150, 8)            32        
_________________________________________________________________
flatten_21 (Flatten)         (None, 1200)              0         
_________________________________________________________________
dense_48 (Dense)             (None, 250)               300250    
__________

In [85]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 23s 1ms/step - loss: 0.8914 - acc: 0.5899 - val_loss: 0.9190 - val_acc: 0.6379
Epoch 2/100
17621/17621 [==============================] - 21s 1ms/step - loss: 0.5089 - acc: 0.7992 - val_loss: 0.5572 - val_acc: 0.7778
Epoch 3/100
17621/17621 [==============================] - 22s 1ms/step - loss: 0.3739 - acc: 0.8541 - val_loss: 0.6372 - val_acc: 0.7692
Epoch 4/100
17621/17621 [==============================] - 22s 1ms/step - loss: 0.3057 - acc: 0.8838 - val_loss: 0.6596 - val_acc: 0.7819
Epoch 5/100
17621/17621 [==============================] - 22s 1ms/step - loss: 0.2686 - acc: 0.8959 - val_loss: 0.6323 - val_acc: 0.7799


In [86]:
#decreased learning rate
from keras import optimizers

model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(Conv1D(filters=8, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Flatten())

model.add(Dense(250, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(3, activation='softmax'))

nadam = optimizers.Nadam (lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 300, 8)            7208      
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 150, 8)            0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 150, 8)            0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 150, 8)            32        
_________________________________________________________________
flatten_22 (Flatten)         (None, 1200)              0         
_________________________________________________________________
dense_50 (Dense)             (None, 250)               300250    
__________

In [87]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 23s 1ms/step - loss: 1.0282 - acc: 0.5460 - val_loss: 0.9622 - val_acc: 0.5904
Epoch 2/100
17621/17621 [==============================] - 22s 1ms/step - loss: 0.5155 - acc: 0.7964 - val_loss: 0.5802 - val_acc: 0.7635
Epoch 3/100
17621/17621 [==============================] - 22s 1ms/step - loss: 0.3561 - acc: 0.8620 - val_loss: 0.5360 - val_acc: 0.7875
Epoch 4/100
17621/17621 [==============================] - 22s 1ms/step - loss: 0.2859 - acc: 0.8897 - val_loss: 0.5617 - val_acc: 0.7794
Epoch 5/100
17621/17621 [==============================] - 22s 1ms/step - loss: 0.2300 - acc: 0.9135 - val_loss: 0.6056 - val_acc: 0.7901
Epoch 6/100
17621/17621 [==============================] - 22s 1ms/step - loss: 0.2016 - acc: 0.9237 - val_loss: 0.6354 - val_acc: 0.7896


In [88]:
# create the model
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(Conv1D(filters=8, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))


model.add(Dense(3, activation='softmax'))


nadam = optimizers.Nadam (lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_28 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 300, 8)            7208      
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 150, 8)            0         
_________________________________________________________________
flatten_23 (Flatten)         (None, 1200)              0         
_________________________________________________________________
dense_52 (Dense)             (None, 250)               300250    
_________________________________________________________________
dense_53 (Dense)             (None, 3)                 753       
Total params: 3,308,211
Trainable params: 3,308,211
Non-trainable params: 0
_________________________________________________________________


In [89]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 23s 1ms/step - loss: 0.7142 - acc: 0.6779 - val_loss: 0.5149 - val_acc: 0.7916
Epoch 2/100
17621/17621 [==============================] - 21s 1ms/step - loss: 0.2950 - acc: 0.8912 - val_loss: 0.5387 - val_acc: 0.7896
Epoch 3/100
17621/17621 [==============================] - 21s 1ms/step - loss: 0.1175 - acc: 0.9598 - val_loss: 0.6866 - val_acc: 0.7896
Epoch 4/100
17621/17621 [==============================] - 21s 1ms/step - loss: 0.0406 - acc: 0.9881 - val_loss: 0.9310 - val_acc: 0.7804


In [90]:
# add one more layer

model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(Conv1D(filters=8, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(200, activation='relu'))

model.add(Dense(3, activation='softmax'))


nadam = optimizers.Nadam (lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_29 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 300, 8)            7208      
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 150, 8)            0         
_________________________________________________________________
flatten_24 (Flatten)         (None, 1200)              0         
_________________________________________________________________
dense_54 (Dense)             (None, 250)               300250    
_________________________________________________________________
dense_55 (Dense)             (None, 200)               50200     
_________________________________________________________________
dense_56 (Dense)             (None, 3)                 603       
Total para

In [91]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 23s 1ms/step - loss: 0.7091 - acc: 0.6802 - val_loss: 0.5367 - val_acc: 0.7906
Epoch 2/100
17621/17621 [==============================] - 21s 1ms/step - loss: 0.2860 - acc: 0.8935 - val_loss: 0.5530 - val_acc: 0.8013
Epoch 3/100
17621/17621 [==============================] - 21s 1ms/step - loss: 0.1135 - acc: 0.9600 - val_loss: 0.7526 - val_acc: 0.7804
Epoch 4/100
17621/17621 [==============================] - 22s 1ms/step - loss: 0.0472 - acc: 0.9836 - val_loss: 1.0210 - val_acc: 0.7819


In [92]:
# change convolution layer to second layer
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(Dense(250, activation='relu'))

model.add(Conv1D(filters=8, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

model.add(Dense(200, activation='relu'))

model.add(Dense(3, activation='softmax'))


nadam = optimizers.Nadam (lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_30 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
dense_57 (Dense)             (None, 300, 250)          75250     
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 300, 8)            6008      
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 150, 8)            0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 1200)              0         
_________________________________________________________________
dense_58 (Dense)             (None, 200)               240200    
_________________________________________________________________
dense_59 (Dense)             (None, 3)                 603       
Total para

In [93]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 31s 2ms/step - loss: 0.7564 - acc: 0.6501 - val_loss: 0.7046 - val_acc: 0.7135
Epoch 2/100
17621/17621 [==============================] - 29s 2ms/step - loss: 0.3657 - acc: 0.8602 - val_loss: 0.5976 - val_acc: 0.7620
Epoch 3/100
17621/17621 [==============================] - 29s 2ms/step - loss: 0.2151 - acc: 0.9187 - val_loss: 0.6215 - val_acc: 0.7835
Epoch 4/100
17621/17621 [==============================] - 29s 2ms/step - loss: 0.1247 - acc: 0.9561 - val_loss: 0.8164 - val_acc: 0.7870
Epoch 5/100
17621/17621 [==============================] - 29s 2ms/step - loss: 0.0695 - acc: 0.9745 - val_loss: 1.0024 - val_acc: 0.7732


In [95]:
## add more convolutional layer
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(Conv1D(filters=8, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())

model.add(Conv1D(filters=16, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

model.add(Dense(250, activation='relu'))
model.add(Dense(200, activation='relu'))

model.add(Dense(3, activation='softmax'))


nadam = optimizers.Nadam (lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_32 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 300, 8)            7208      
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 150, 8)            0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 150, 16)           400       
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 75, 16)            0         
_________________________________________________________________
flatten_27 (Flatten)         (None, 1200)              0         
_________________________________________________________________
dense_60 (Dense)             (None, 250)               300250    
__________

In [96]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 24s 1ms/step - loss: 0.8040 - acc: 0.6194 - val_loss: 0.5769 - val_acc: 0.7697
Epoch 2/100
17621/17621 [==============================] - 22s 1ms/step - loss: 0.3582 - acc: 0.8628 - val_loss: 0.5707 - val_acc: 0.7804
Epoch 3/100
17621/17621 [==============================] - 21s 1ms/step - loss: 0.1485 - acc: 0.9470 - val_loss: 0.6926 - val_acc: 0.7753
Epoch 4/100
17621/17621 [==============================] - 22s 1ms/step - loss: 0.0668 - acc: 0.9771 - val_loss: 1.0711 - val_acc: 0.7492
Epoch 5/100
17621/17621 [==============================] - 22s 1ms/step - loss: 0.0422 - acc: 0.9855 - val_loss: 1.3867 - val_acc: 0.7457


In [97]:
# keeping simpler model but increasing the number of filters to learn
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

model.add(Dense(250, activation='relu'))
#model.add(Dense(200, activation='relu'))

model.add(Dense(3, activation='softmax'))


nadam = optimizers.Nadam (lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_33 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 300, 32)           28832     
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 150, 32)           0         
_________________________________________________________________
flatten_28 (Flatten)         (None, 4800)              0         
_________________________________________________________________
dense_63 (Dense)             (None, 250)               1200250   
_________________________________________________________________
dense_64 (Dense)             (None, 3)                 753       
Total params: 4,229,835
Trainable params: 4,229,835
Non-trainable params: 0
_________________________________________________________________


In [98]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 27s 2ms/step - loss: 0.6978 - acc: 0.6859 - val_loss: 0.5246 - val_acc: 0.7722
Epoch 2/100
17621/17621 [==============================] - 25s 1ms/step - loss: 0.2997 - acc: 0.8871 - val_loss: 0.5388 - val_acc: 0.7916
Epoch 3/100
17621/17621 [==============================] - 25s 1ms/step - loss: 0.1167 - acc: 0.9612 - val_loss: 0.6537 - val_acc: 0.7937
Epoch 4/100
17621/17621 [==============================] - 25s 1ms/step - loss: 0.0310 - acc: 0.9920 - val_loss: 0.9063 - val_acc: 0.7891


In [99]:
#Now add one more layer of convolution with half number of filters

model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=16, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

model.add(Dense(250, activation='relu'))
#model.add(Dense(200, activation='relu'))

model.add(Dense(3, activation='softmax'))


nadam = optimizers.Nadam (lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_34 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 300, 32)           28832     
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 150, 32)           0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 150, 16)           1552      
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 75, 16)            0         
_________________________________________________________________
flatten_29 (Flatten)         (None, 1200)              0         
_________________________________________________________________
dense_65 (Dense)             (None, 250)               300250    
__________

In [100]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 25s 1ms/step - loss: 0.7305 - acc: 0.6701 - val_loss: 0.5203 - val_acc: 0.7809
Epoch 2/100
17621/17621 [==============================] - 24s 1ms/step - loss: 0.3118 - acc: 0.8818 - val_loss: 0.5103 - val_acc: 0.7967
Epoch 3/100
17621/17621 [==============================] - 24s 1ms/step - loss: 0.1313 - acc: 0.9540 - val_loss: 0.6437 - val_acc: 0.7952
Epoch 4/100
17621/17621 [==============================] - 24s 1ms/step - loss: 0.0460 - acc: 0.9855 - val_loss: 0.9382 - val_acc: 0.7707
Epoch 5/100
17621/17621 [==============================] - 24s 1ms/step - loss: 0.0144 - acc: 0.9965 - val_loss: 0.9863 - val_acc: 0.7998


In [101]:
# KEEP simpler model but don't use max pooling
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Flatten())

model.add(Dense(250, activation='relu'))
#model.add(Dense(200, activation='relu'))

model.add(Dense(3, activation='softmax'))


nadam = optimizers.Nadam (lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_35 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 300, 32)           28832     
_________________________________________________________________
flatten_30 (Flatten)         (None, 9600)              0         
_________________________________________________________________
dense_67 (Dense)             (None, 250)               2400250   
_________________________________________________________________
dense_68 (Dense)             (None, 3)                 753       
Total params: 5,429,835
Trainable params: 5,429,835
Non-trainable params: 0
_________________________________________________________________
None


In [102]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 30s 2ms/step - loss: 0.7005 - acc: 0.6817 - val_loss: 0.5143 - val_acc: 0.7891
Epoch 2/100
17621/17621 [==============================] - 28s 2ms/step - loss: 0.3088 - acc: 0.8818 - val_loss: 0.5217 - val_acc: 0.7988
Epoch 3/100
17621/17621 [==============================] - 28s 2ms/step - loss: 0.1246 - acc: 0.9560 - val_loss: 0.6964 - val_acc: 0.7967
Epoch 4/100
17621/17621 [==============================] - 28s 2ms/step - loss: 0.0317 - acc: 0.9915 - val_loss: 0.9792 - val_acc: 0.7824


In [10]:
## Now using two convolutional layers without using maxpooling

# KEEP simpler model but don't use max pooling
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(Conv1D(filters=16, kernel_size=3, padding='same', activation='relu'))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))

model.add(Flatten())

model.add(Dense(250, activation='relu'))
#model.add(Dense(200, activation='relu'))

model.add(Dense(3, activation='softmax'))


nadam = optimizers.Nadam (lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 300)          3000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 300, 16)           14416     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 300, 32)           1568      
_________________________________________________________________
flatten_1 (Flatten)          (None, 9600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               2400250   
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 753       
Total params: 5,416,987
Trainable params: 5,416,987
Non-trainable params: 0
_________________________________________________________________


In [11]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 40s 2ms/step - loss: 0.7099 - acc: 0.6809 - val_loss: 0.5224 - val_acc: 0.7835
Epoch 2/100
17621/17621 [==============================] - 29s 2ms/step - loss: 0.3263 - acc: 0.8751 - val_loss: 0.5226 - val_acc: 0.7957
Epoch 3/100
17621/17621 [==============================] - 28s 2ms/step - loss: 0.1602 - acc: 0.9420 - val_loss: 0.7958 - val_acc: 0.7702
Epoch 4/100
17621/17621 [==============================] - 29s 2ms/step - loss: 0.0665 - acc: 0.9771 - val_loss: 1.2759 - val_acc: 0.7375


In [15]:
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(Dense(300, activation='relu'))

model.add(Conv1D(filters=64, kernel_size=5, padding='same', activation='relu'))

model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'))

model.add(Flatten())

model.add(Dense(250, activation='relu'))
#model.add(Dense(200, activation='relu'))

model.add(Dense(3, activation='softmax'))


nadam = optimizers.Nadam (lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 300, 300)          3000000   
_________________________________________________________________
dense_6 (Dense)              (None, 300, 300)          90300     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 300, 64)           96064     
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 300, 32)           10272     
_________________________________________________________________
flatten_5 (Flatten)          (None, 9600)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 250)               2400250   
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 753       
Total para

In [16]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 56s 3ms/step - loss: 0.7868 - acc: 0.6270 - val_loss: 0.5483 - val_acc: 0.7707
Epoch 2/100
17621/17621 [==============================] - 55s 3ms/step - loss: 0.3864 - acc: 0.8505 - val_loss: 0.5379 - val_acc: 0.7768
Epoch 3/100
17621/17621 [==============================] - 55s 3ms/step - loss: 0.2505 - acc: 0.9080 - val_loss: 0.5791 - val_acc: 0.7773
Epoch 4/100
17621/17621 [==============================] - 55s 3ms/step - loss: 0.1757 - acc: 0.9345 - val_loss: 0.7585 - val_acc: 0.7702
Epoch 5/100
17621/17621 [==============================] - 55s 3ms/step - loss: 0.1362 - acc: 0.9511 - val_loss: 0.8404 - val_acc: 0.7646


In [17]:
# Using different kernel size
# KEEP simpler model but don't use max pooling
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'))
model.add(Flatten())

model.add(Dense(250, activation='relu'))
#model.add(Dense(200, activation='relu'))

model.add(Dense(3, activation='softmax'))


nadam = optimizers.Nadam (lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 300, 300)          3000000   
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 300, 32)           48032     
_________________________________________________________________
flatten_6 (Flatten)          (None, 9600)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 250)               2400250   
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 753       
Total params: 5,449,035
Trainable params: 5,449,035
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 31s 2ms/step - loss: 0.7136 - acc: 0.6813 - val_loss: 0.4958 - val_acc: 0.7937
Epoch 2/100
17621/17621 [==============================] - 31s 2ms/step - loss: 0.3169 - acc: 0.8772 - val_loss: 0.5449 - val_acc: 0.7926
Epoch 3/100
17621/17621 [==============================] - 31s 2ms/step - loss: 0.1331 - acc: 0.9541 - val_loss: 0.7122 - val_acc: 0.7870
Epoch 4/100
17621/17621 [==============================] - 31s 2ms/step - loss: 0.0343 - acc: 0.9895 - val_loss: 0.9415 - val_acc: 0.7932


In [19]:
# Using different kernel size
# KEEP simpler model but don't use max pooling
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'))
model.add(Conv1D(filters=64, kernel_size=5, padding='same', activation='relu'))
model.add(Flatten())

model.add(Dense(250, activation='relu'))
#model.add(Dense(200, activation='relu'))

model.add(Dense(3, activation='softmax'))


nadam = optimizers.Nadam (lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 300, 300)          3000000   
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 300, 32)           48032     
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 300, 64)           10304     
_________________________________________________________________
flatten_7 (Flatten)          (None, 19200)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 250)               4800250   
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 753       
Total params: 7,859,339
Trainable params: 7,859,339
Non-trainable params: 0
_________________________________________________________________


In [20]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 42s 2ms/step - loss: 0.7186 - acc: 0.6761 - val_loss: 0.5270 - val_acc: 0.7809
Epoch 2/100
17621/17621 [==============================] - 41s 2ms/step - loss: 0.3400 - acc: 0.8706 - val_loss: 0.5205 - val_acc: 0.8029
Epoch 3/100
17621/17621 [==============================] - 41s 2ms/step - loss: 0.1880 - acc: 0.9310 - val_loss: 0.5984 - val_acc: 0.7906
Epoch 4/100
17621/17621 [==============================] - 41s 2ms/step - loss: 0.0964 - acc: 0.9646 - val_loss: 0.7995 - val_acc: 0.7906
Epoch 5/100
17621/17621 [==============================] - 41s 2ms/step - loss: 0.0575 - acc: 0.9814 - val_loss: 1.1407 - val_acc: 0.7732


In [22]:
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())

model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'))
#model.add(Flatten())

model.add(Dense(400, activation='relu'))

model.add(Conv1D(filters=64, kernel_size=5, padding='same', activation='relu'))
model.add(Flatten())

model.add(Dense(250, activation='relu'))
#model.add(Dense(200, activation='relu'))

model.add(Dense(3, activation='softmax'))


nadam = optimizers.Nadam (lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 300, 300)          3000000   
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 300, 32)           48032     
_________________________________________________________________
dense_14 (Dense)             (None, 300, 400)          13200     
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 300, 64)           128064    
_________________________________________________________________
flatten_9 (Flatten)          (None, 19200)             0         
_________________________________________________________________
dense_15 (Dense)             (None, 250)               4800250   
_________________________________________________________________
dense_16 (Dense)             (None, 3)                 753       
Total para

In [23]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 75s 4ms/step - loss: 0.8777 - acc: 0.5714 - val_loss: 0.5831 - val_acc: 0.7671
Epoch 2/100
17621/17621 [==============================] - 74s 4ms/step - loss: 0.4025 - acc: 0.8422 - val_loss: 0.5307 - val_acc: 0.7783
Epoch 3/100
17621/17621 [==============================] - 74s 4ms/step - loss: 0.2125 - acc: 0.9220 - val_loss: 0.7077 - val_acc: 0.7768
Epoch 4/100
17621/17621 [==============================] - 74s 4ms/step - loss: 0.1101 - acc: 0.9606 - val_loss: 1.0982 - val_acc: 0.7288
Epoch 5/100
17621/17621 [==============================] - 74s 4ms/step - loss: 0.0637 - acc: 0.9787 - val_loss: 1.0690 - val_acc: 0.7799


In [29]:
#lstm with cnn
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_length))
#model.add(Flatten())
model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'))

model.add(LSTM(100))
model.add(BatchNormalization())



model.add(Dense(1024, activation='relu'))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 300, 300)          3000000   
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 300, 32)           48032     
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               53200     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_17 (Dense)             (None, 1024)              103424    
_________________________________________________________________
dense_18 (Dense)             (None, 3)                 3075      
_________________________________________________________________
activation_1 (Activation)    (None, 3)                 0         
Total para

In [30]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 142s 8ms/step - loss: 1.0887 - val_loss: 1.0887
Epoch 2/100
17621/17621 [==============================] - 142s 8ms/step - loss: 1.0879 - val_loss: 5.3173
Epoch 3/100
17621/17621 [==============================] - 143s 8ms/step - loss: 1.0883 - val_loss: 9.5281
Epoch 4/100
17621/17621 [==============================] - 145s 8ms/step - loss: 1.0878 - val_loss: 1.5347


In [37]:
## Create a functional api for shared features from cnn and lstm

#lstm with cnn
from keras.layers import Input, Embedding, LSTM, Dense, concatenate
from keras.models import Model

# Headline input: meant to receive sequences of 100 integers, between 1 and 10000.
# Note that we can name any layer by passing it a "name" argument.
main_input = Input(shape=(300,), dtype='int32', name='main_input')

#question_input = Input(shape=(100,), dtype='int32')
#embedded_question = Embedding(input_dim=10000, output_dim=256, input_length=100)(question_input)
#encoded_question = LSTM(256)(embedded_question)

# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
#x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)

# A LSTM will transform the vector sequence into a single vector,
# containing information about the entire sequence
#lstm_out = LSTM(32)(x)
# vocab_size = 10000, maximum_length = 300
x = Embedding(vocab_size, 300, input_length=max_length)(main_input)
#model.add(Flatten())
lstm_encoding = LSTM(100)(x)


cnn_mod = Sequential()
cnn_mod.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu', input_shape=(300,300)))
cnn_mod.add(Flatten())
cnn_encoding = cnn_mod(x)




merged = concatenate([lstm_encoding, cnn_encoding])

#batch_normalized = BatchNormalization()(merged)



hidden1 = Dense(1024, activation='relu')(merged)

#hidden2 = Dense(3)(hidden1)
output = Dense(3, activation = 'softmax')(hidden1)


model = Model(inputs = main_input, outputs = output)

model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_21 (Embedding)        (None, 300, 300)     3000000     main_input[0][0]                 
__________________________________________________________________________________________________
lstm_12 (LSTM)                  (None, 100)          160400      embedding_21[0][0]               
__________________________________________________________________________________________________
sequential_18 (Sequential)      (None, 9600)         48032       embedding_21[0][0]               
__________________________________________________________________________________________________
concatenat

In [38]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(padded_docs_train, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(padded_docs_valid, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 263s 15ms/step - loss: 0.7126 - val_loss: 0.5137
Epoch 2/100
17621/17621 [==============================] - 261s 15ms/step - loss: 0.3072 - val_loss: 0.5431
Epoch 3/100
17621/17621 [==============================] - 262s 15ms/step - loss: 0.1026 - val_loss: 0.7131
Epoch 4/100
17621/17621 [==============================] - 268s 15ms/step - loss: 0.0260 - val_loss: 0.9948


In [43]:
import gc
#del model
gc.collect()

926

In [ ]:
##How to run multiple times and average the results
##How to use cross validation
##How to do model stacking
##How to use abhishek's model stacking
##be able to create a cnn layer and lstm together from input and then pass it to a dense layer
